## Project 3 - Wildfires

A wildfire is an uncontrolled fire that burns in the wildland vegetation, including rural areas, shrublands/grasslands, savannas, croplands, etc. We will be using datasets that will help us identify how much area is burnt every week per country, the total land area burnt per year, as well as the type of land (ecosystem) burnt per year. The data we will be using were taken from the years 2012-2022.


In [143]:
import pandas as pd
from pathlib import Path
import hvplot.pandas
import matplotlib.pyplot as plt
import scipy.stats as st
import numpy as np
from scipy.stats import linregress, pearsonr
import requests
import time
from pathlib import Path
import folium
from ipywidgets import interact, Dropdown, Output
from geopy.geocoders import Nominatim

In [144]:
# Create a path to CSV files, read the path and print DataFrame for the first CSV file
file_path = Path("Project3/1-cumulative-area-burnt-by-wildfires-by-week.csv")
wildfires = pd.read_csv(file_path)
wildfires

,Entity,Code,Year,Cumulative area burnt by wildfires in 2024,Cumulative area burnt by wildfires in 2023,Cumulative area burnt by wildfires in 2022,Cumulative area burnt by wildfires in 2021,Cumulative area burnt by wildfires in 2020,Cumulative area burnt by wildfires in 2019,Cumulative area burnt by wildfires in 2018,Cumulative area burnt by wildfires in 2017,Cumulative area burnt by wildfires in 2016,Cumulative area burnt by wildfires in 2015,Cumulative area burnt by wildfires in 2014,Cumulative area burnt by wildfires in 2013,Cumulative area burnt by wildfires in 2012
0,Afghanistan,AFG,1,391.0,0,0,1246,0,0,117,0,0,368,0,0,0
1,Afghanistan,AFG,2,559.0,0,0,2680,963,20,117,82,0,425,426,0,0
2,Afghanistan,AFG,3,841.0,0,0,3539,1181,20,264,129,390,425,426,54,0
3,Afghanistan,AFG,4,841.0,0,0,3719,2030,20,411,129,390,425,426,362,368
4,Afghanistan,AFG,5,841.0,0,215,4131,3351,20,411,129,1043,425,426,379,597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13411,Zimbabwe,ZWE,48,NaN,2358715,3017486,2090438,1221386,2752906,2705357,2437024,2176472,3182494,2831917,2556610,3266499
13412,Zimbabwe,ZWE,49,NaN,2383435,3017486,2108381,1221386,2753700,2705357,2441223,2178815,3192311,2861358,2562822,3269141
13413,Zimbabwe,ZWE,50,NaN,2385610,3017694,2113226,1221386,2754152,2707267,2443004,2179147,3194733,2862483,2564134,3270515
13414,Zimbabwe,ZWE,51,NaN,2385610,3017694,2113226,1221548,2756253,2709955,2443661,2179147,3195377,2862758,2564322,3271228


In [145]:
#Drop unneeded columns
wildfires_clean = wildfires.drop(columns=['Cumulative area burnt by wildfires in 2024', 'Cumulative area burnt by wildfires in 2023'])
wildfires_clean

,Entity,Code,Year,Cumulative area burnt by wildfires in 2022,Cumulative area burnt by wildfires in 2021,Cumulative area burnt by wildfires in 2020,Cumulative area burnt by wildfires in 2019,Cumulative area burnt by wildfires in 2018,Cumulative area burnt by wildfires in 2017,Cumulative area burnt by wildfires in 2016,Cumulative area burnt by wildfires in 2015,Cumulative area burnt by wildfires in 2014,Cumulative area burnt by wildfires in 2013,Cumulative area burnt by wildfires in 2012
0,Afghanistan,AFG,1,0,1246,0,0,117,0,0,368,0,0,0
1,Afghanistan,AFG,2,0,2680,963,20,117,82,0,425,426,0,0
2,Afghanistan,AFG,3,0,3539,1181,20,264,129,390,425,426,54,0
3,Afghanistan,AFG,4,0,3719,2030,20,411,129,390,425,426,362,368
4,Afghanistan,AFG,5,215,4131,3351,20,411,129,1043,425,426,379,597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13411,Zimbabwe,ZWE,48,3017486,2090438,1221386,2752906,2705357,2437024,2176472,3182494,2831917,2556610,3266499
13412,Zimbabwe,ZWE,49,3017486,2108381,1221386,2753700,2705357,2441223,2178815,3192311,2861358,2562822,3269141
13413,Zimbabwe,ZWE,50,3017694,2113226,1221386,2754152,2707267,2443004,2179147,3194733,2862483,2564134,3270515
13414,Zimbabwe,ZWE,51,3017694,2113226,1221548,2756253,2709955,2443661,2179147,3195377,2862758,2564322,3271228


In [146]:
# Rename columns 
area_burnt_weekly = wildfires_clean.rename(columns={"Entity":"Country", "Year":"Week"})
area_burnt_weekly

,Country,Code,Week,Cumulative area burnt by wildfires in 2022,Cumulative area burnt by wildfires in 2021,Cumulative area burnt by wildfires in 2020,Cumulative area burnt by wildfires in 2019,Cumulative area burnt by wildfires in 2018,Cumulative area burnt by wildfires in 2017,Cumulative area burnt by wildfires in 2016,Cumulative area burnt by wildfires in 2015,Cumulative area burnt by wildfires in 2014,Cumulative area burnt by wildfires in 2013,Cumulative area burnt by wildfires in 2012
0,Afghanistan,AFG,1,0,1246,0,0,117,0,0,368,0,0,0
1,Afghanistan,AFG,2,0,2680,963,20,117,82,0,425,426,0,0
2,Afghanistan,AFG,3,0,3539,1181,20,264,129,390,425,426,54,0
3,Afghanistan,AFG,4,0,3719,2030,20,411,129,390,425,426,362,368
4,Afghanistan,AFG,5,215,4131,3351,20,411,129,1043,425,426,379,597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13411,Zimbabwe,ZWE,48,3017486,2090438,1221386,2752906,2705357,2437024,2176472,3182494,2831917,2556610,3266499
13412,Zimbabwe,ZWE,49,3017486,2108381,1221386,2753700,2705357,2441223,2178815,3192311,2861358,2562822,3269141
13413,Zimbabwe,ZWE,50,3017694,2113226,1221386,2754152,2707267,2443004,2179147,3194733,2862483,2564134,3270515
13414,Zimbabwe,ZWE,51,3017694,2113226,1221548,2756253,2709955,2443661,2179147,3195377,2862758,2564322,3271228


In [147]:
# Create a path to CSV files, read the path and print DataFrame for the second CSV file
file_path2 = ("Project3/3-share-of-the-total-land-area-burnt-by-wildfires-each-year.csv")
wildfires2 = pd.read_csv(file_path2)
wildfires2

,Entity,Code,Year,Annual share of the total land area burnt by wildfires
0,Afghanistan,AFG,2012,0.015976
1,Afghanistan,AFG,2013,0.009025
2,Afghanistan,AFG,2014,0.007864
3,Afghanistan,AFG,2015,0.006256
4,Afghanistan,AFG,2016,0.014829
...,...,...,...,...
3141,Zimbabwe,ZWE,2020,3.126520
3142,Zimbabwe,ZWE,2021,5.408635
3143,Zimbabwe,ZWE,2022,7.725561
3144,Zimbabwe,ZWE,2023,6.105052


In [148]:
# Rename the columns
wildfires2_rename = wildfires2.rename(columns={"Entity":"Country"})
wildfires2_rename

,Country,Code,Year,Annual share of the total land area burnt by wildfires
0,Afghanistan,AFG,2012,0.015976
1,Afghanistan,AFG,2013,0.009025
2,Afghanistan,AFG,2014,0.007864
3,Afghanistan,AFG,2015,0.006256
4,Afghanistan,AFG,2016,0.014829
...,...,...,...,...
3141,Zimbabwe,ZWE,2020,3.126520
3142,Zimbabwe,ZWE,2021,5.408635
3143,Zimbabwe,ZWE,2022,7.725561
3144,Zimbabwe,ZWE,2023,6.105052


In [149]:
# Remove 2023 and 2024, as we are only using data from 2012-2022
area_burnt_yearly = wildfires2.drop(wildfires2[wildfires2['Year'] > 2022].index)
area_burnt_yearly

,Entity,Code,Year,Annual share of the total land area burnt by wildfires
0,Afghanistan,AFG,2012,0.015976
1,Afghanistan,AFG,2013,0.009025
2,Afghanistan,AFG,2014,0.007864
3,Afghanistan,AFG,2015,0.006256
4,Afghanistan,AFG,2016,0.014829
...,...,...,...,...
3139,Zimbabwe,ZWE,2018,6.937745
3140,Zimbabwe,ZWE,2019,7.056078
3141,Zimbabwe,ZWE,2020,3.126520
3142,Zimbabwe,ZWE,2021,5.408635


In [150]:
# Export area_burnt_weekly as a CSV file.
area_burnt_weekly.to_csv("/Users/artemzalivchiy/Desktop/Project3/area_burnt_weekly.csv", index=False)

In [151]:
# Create a path to CSV files, read the path and print DataFrame for the third CSV file
file_path3 = ("Project3/5-annual-burned-area-by-landcover.csv")
wildfires3 = pd.read_csv(file_path3)
wildfires3

,Entity,Code,Year,Yearly burned area across other land categories,Yearly burned area across croplands,Yearly burned area across forests,Yearly burned area across savannas,Yearly burned area across shrublands and grasslands
0,Afghanistan,AFG,2002,0.00000,14167.473,0.0000,0.000,18052.793
1,Afghanistan,AFG,2003,1309.41800,21680.530,0.0000,0.000,112030.360
2,Afghanistan,AFG,2004,407.85200,4357.571,0.0000,0.000,24556.953
3,Afghanistan,AFG,2005,85.86400,1395.281,0.0000,64.398,10883.193
4,Afghanistan,AFG,2006,1116.22690,9809.900,0.0000,64.398,7169.603
...,...,...,...,...,...,...,...,...
4930,Zimbabwe,ZWE,2018,837.17000,345579.000,36449.0430,420537.800,1806674.800
4931,Zimbabwe,ZWE,2019,472.24902,344205.200,28012.9590,357277.900,1784092.600
4932,Zimbabwe,ZWE,2020,1180.62290,248317.160,8006.7695,96081.230,643503.750
4933,Zimbabwe,ZWE,2021,364.92000,292214.840,2897.8940,155262.620,1353895.200


In [152]:
# Rename the columns
wildfires3_rename = wildfires3.rename(columns={"Entity":"Country"})
wildfires3_rename

,Country,Code,Year,Yearly burned area across other land categories,Yearly burned area across croplands,Yearly burned area across forests,Yearly burned area across savannas,Yearly burned area across shrublands and grasslands
0,Afghanistan,AFG,2002,0.00000,14167.473,0.0000,0.000,18052.793
1,Afghanistan,AFG,2003,1309.41800,21680.530,0.0000,0.000,112030.360
2,Afghanistan,AFG,2004,407.85200,4357.571,0.0000,0.000,24556.953
3,Afghanistan,AFG,2005,85.86400,1395.281,0.0000,64.398,10883.193
4,Afghanistan,AFG,2006,1116.22690,9809.900,0.0000,64.398,7169.603
...,...,...,...,...,...,...,...,...
4930,Zimbabwe,ZWE,2018,837.17000,345579.000,36449.0430,420537.800,1806674.800
4931,Zimbabwe,ZWE,2019,472.24902,344205.200,28012.9590,357277.900,1784092.600
4932,Zimbabwe,ZWE,2020,1180.62290,248317.160,8006.7695,96081.230,643503.750
4933,Zimbabwe,ZWE,2021,364.92000,292214.840,2897.8940,155262.620,1353895.200


In [153]:
# Remove 2002-2011, as we are only using data from 2012-2022
area_burnt_landcover = wildfires3_rename.drop(wildfires3_rename[wildfires3_rename['Year'] < 2012].index)
area_burnt_landcover

,Country,Code,Year,Yearly burned area across other land categories,Yearly burned area across croplands,Yearly burned area across forests,Yearly burned area across savannas,Yearly burned area across shrublands and grasslands
10,Afghanistan,AFG,2012,0.00000,0.00000,0.0000,0.000,5.559661e+03
11,Afghanistan,AFG,2013,364.91900,837.16800,0.0000,0.000,5.946045e+03
12,Afghanistan,AFG,2014,64.39800,150.26100,0.0000,0.000,9.659640e+02
13,Afghanistan,AFG,2015,171.72700,493.71500,0.0000,0.000,1.481145e+03
14,Afghanistan,AFG,2016,407.85100,601.04504,0.0000,300.523,3.091086e+03
...,...,...,...,...,...,...,...,...
4930,Zimbabwe,ZWE,2018,837.17000,345579.00000,36449.0430,420537.800,1.806675e+06
4931,Zimbabwe,ZWE,2019,472.24902,344205.20000,28012.9590,357277.900,1.784093e+06
4932,Zimbabwe,ZWE,2020,1180.62290,248317.16000,8006.7695,96081.230,6.435038e+05
4933,Zimbabwe,ZWE,2021,364.92000,292214.84000,2897.8940,155262.620,1.353895e+06


In [159]:
data = pd.read_csv('/Users/artemzalivchiy/Desktop/Project3/area_burnt_weekly.csv')
geolocator = Nominatim(user_agent="wildfire_map")

# Function to update the map based on dropdown selection
def update_map(country, week, year):
    # Filter data based on selected country, week, and year
    filtered_data = data[(data['Country'] == country) & (data['Week'] == week)]

    # Geocode the selected country to get its coordinates
    location = geolocator.geocode(country)
    if location:
        country_center = [location.latitude, location.longitude]
    else:
        country_center = [0, 0]  # Default to center of the map

    # Initialize a Folium map centered on the selected country
    my_map = folium.Map(location=country_center, zoom_start=4)

    # Add Markers for the selected country, week, and year
    for index, row in filtered_data.iterrows():
        popup_text = f"<b>{country}</b><br>"
        popup_text += f"Week: {week}<br>"
        popup_text += f"Wildfire area {year} Hectares: {row[f'Cumulative area burnt by wildfires in {year}']}<br>"
        folium.Marker(location=country_center, popup=popup_text).add_to(my_map)

    # Display the map
    display(my_map)

# Get unique country names and weeks
country_names = data['Country'].unique().tolist()
weeks = data['Week'].unique().tolist()
years = [int(col.split()[-1]) for col in data.columns if col.startswith('Cumulative area burnt by wildfires')]

# Create dropdown widgets
country_dropdown = Dropdown(options=country_names, description='Country:')
week_dropdown = Dropdown(options=weeks, description='Week:')
year_dropdown = Dropdown(options=years, description='Year:')

# Display interactive dropdowns
interact(update_map, country=country_dropdown, week=week_dropdown, year=year_dropdown);

interactive(children=(Dropdown(description='Country:', options=('Afghanistan', 'Africa', 'Akrotiri and Dhekeli…